In [5]:
import threading

In [39]:
class Agent:
    # size of the puzzle matrix
    grid_size = None
    def __init__(self, name, current_position, target_position, next_position):
        self.name = name
        self.current_position = current_position
        self.target_position = target_position
        self.next_position = next_position
        
    def describe(self):
        print(f"Agent {self.name} is at {self.current_position} and wants to go to {self.target_position}. The next position is {self.next_position}.") 
         
    # def to show grid of the game 
    def Grid(game="X"):
        # First row
        for j in range(Agent.grid_size):
            print(f"| {j+1} ", end='')
        print("| ")
        print((Agent.grid_size*4+1)*"-")

        # Other rows
        for i in range(Agent.grid_size):
            # print(f"| {i+1} ", end='')
            for j in range(Agent.grid_size):
                 print(f"| {game} ", end='')
            print("| ")
            print((Agent.grid_size*4+1)*"-")


        

In [43]:
Agent.grid_size=3
Agent.Grid()

| 1 | 2 | 3 | 
-------------
| X | X | X | 
-------------
| X | X | X | 
-------------
| X | X | X | 
-------------


In [7]:
agent = Agent("A", (1, 0), (1, 1), (3, 3))